# Day 1 - 提示工程

歡迎參加 Kaggle 五天生成式 AI 課程！

這份筆記本將帶你入門 Gemini API，並逐步介紹教材中提及的各種提示範例與技巧，這些內容在提示工程白皮書中也有詳細說明。雖然你不一定要先閱讀白皮書才能使用這份筆記本，但它們能為你提供理論背景與補充說明。

## 開始前

在這份筆記本中，你將開始使用 Python SDK 與 AI Studio 來探索提示工程。為了激發靈感，你也可以試試使用 Gemini 模型家族打造的應用程式，例如：

* [TextFX](https://textfx.withgoogle.com/)：與 Lupe Fiasco 合作打造的 AI 工具組，專為饒舌創作設計。
* [SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/)：示範如何利用 Gemini API 與資料庫進行直接對話。
* [NotebookLM](https://notebooklm.google/)：利用 Gemini 模型打造你的個人 AI 研究助理。

## 求助說明

若遇到任何問題，請參考 [FAQ 與疑難排解指南](https://www.kaggle.com/code/markishere/day-0-troubleshooting-and-faqs)。

## Gemini 2.0 新功能！

本課程教材最初於 2024 年 11 月推出，但因為 AI 與大型語言模型發展迅速，我們已更新教材以採用最新模型與功能：

* 本 codelab 已改用 Gemini 2.0 模型家族。
* Python SDK 從 `google-generativeai` 升級為全新統一的 [`google-genai`](https://pypi.org/project/google-genai) SDK。
  * 新 SDK 同時支援開發者版 Gemini API 與 Google Cloud Vertex AI，切換非常簡單（只需修改部分欄位，詳情請參閱相關說明）。
* 新的模型功能已加入，例如本筆記本中介紹的「思考模式」。
* Day 1 亦新增了 [Evaluation codelab](https://www.kaggle.com/code/markishere/day-1-evaluation-and-structured-output)。

## 開始使用 Kaggle 筆記本

如果你是第一次使用 Kaggle 筆記本，歡迎加入！你可以參考 [Kaggle 筆記本說明文件](https://www.kaggle.com/docs/notebooks) 瞭解如何操作。

首先，你需要在 kaggle.com/settings 進行手機驗證。

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_0.png)

接著，點選右上角的 `Copy and Edit` 按鈕，將筆記本複製為可編輯的私人副本，如下圖所示：

![Copy and Edit 按鈕](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_1.png)

複製後，每個程式碼區塊旁都會出現 ▶️ **Run** 按鈕，你可以依序從上到下執行，也可新增區塊自行試驗。如果遇到卡住的情況，可試試 `Factory reset` 選項，或回到原始筆記本再複製一份。

![Run cell 按鈕](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_2.png)

### 問題討論

若有任何問題，請前往 [Kaggle Discord](https://discord.com/invite/kaggle)，到 [`#5dgai-q-and-a` 頻道](https://discord.com/channels/1101210829807956100/1303438695143178251) 尋求協助。

## 開始使用 Gemini API

本筆記本中的所有練習都將使用 [Gemini API](https://ai.google.dev/gemini-api/)，透過 [Python SDK](https://pypi.org/project/google-genai/) 來操作。每個提示也可以直接在 [Google AI Studio](https://aistudio.google.com/) 上試用；若你偏好使用網頁介面，請點選各提示旁的 <img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> AI Studio 連結。

接下來，請將你的 API 金鑰加入 Kaggle 筆記本，作為 Kaggle 使用者密鑰。

![](https://storage.googleapis.com/kaggle-media/Images/5dgai_1.png)  
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_2.png)  
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_3.png)  
![](https://storage.googleapis.com/kaggle-media/Images/5dgai_4.png)

### 安裝 SDK


In [2]:
!pip uninstall -qqy jupyterlab  # 移除 Kaggle 基底映像檔中可能衝突的套件
!pip install -U -q "google-genai==1.7.0"

接著引入 SDK 與輔助模組用來呈現輸出：

In [3]:
from google import genai
from google.genai import types

from IPython.display import HTML, Markdown, display

建立重試機制，讓你可以使用「全部執行」而不用擔心每分鐘配額問題：

In [7]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

### 設定 API 金鑰

請先將你的 API 金鑰新增為 Kaggle Secret，名稱為 `GOOGLE_API_KEY`。（詳細說明請參考 [Kaggle Secret 說明](https://www.kaggle.com/discussions/product-feedback/114053)）

若你尚未擁有 API 金鑰，請至 [AI Studio](https://aistudio.google.com/app/apikey) 取得，詳情請參閱 [Gemini API 文件](https://ai.google.dev/gemini-api/docs/api-key)。


In [9]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

若出現 `No user secrets exist for kernel id ...` 的錯誤，請依照說明透過 `Add-ons` → `Secrets` 新增並啟用金鑰。

![啟用 GOOGLE_API_KEY 的截圖](https://storage.googleapis.com/kaggle-media/Images/5gdai_sc_3.png)

### 執行第一個提問

請測試 API 金鑰設定是否正確，並發送一個請求。

SDK 使用 [`Client` 物件](https://googleapis.github.io/python-genai/genai.html#genai.client.Client) 來發送請求，此物件可控制使用 Gemini API 或 Vertex AI 並處理身份驗證。

本範例選用 `gemini-2.0-flash` 模型。

**注意**：若出現 `TransportError`，請試著執行一次 **Factory reset**。

In [10]:
client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="請像對小孩一樣，簡單地解釋什麼是 AI。")

print(response.text)

想像一下，你有一個超級聰明的玩具機器人。

這個玩具機器人可以：

* **玩遊戲：** 只要你告訴它遊戲規則，它就能自己玩，而且會越玩越厲害！
* **畫畫：** 你可以告訴它你想畫什麼，它就會用你教它的方法畫出來。
* **聽你說話：** 你說話它能聽懂，然後回答你的問題。

這個超級聰明的玩具機器人就是用了 AI！

**AI 就是讓電腦像人一樣思考和學習的技術，讓它們可以做一些原本只有人才能做的事情。**

就好像教玩具機器人，讓它越來越聰明，越來越能幫你忙！



執行結果通常以 Markdown 格式返回，你也可以直接渲染：

In [11]:
Markdown(response.text)

想像一下，你有一個超級聰明的玩具機器人。

這個玩具機器人可以：

* **玩遊戲：** 只要你告訴它遊戲規則，它就能自己玩，而且會越玩越厲害！
* **畫畫：** 你可以告訴它你想畫什麼，它就會用你教它的方法畫出來。
* **聽你說話：** 你說話它能聽懂，然後回答你的問題。

這個超級聰明的玩具機器人就是用了 AI！

**AI 就是讓電腦像人一樣思考和學習的技術，讓它們可以做一些原本只有人才能做的事情。**

就好像教玩具機器人，讓它越來越聰明，越來越能幫你忙！


### 開啟聊天模式

上例使用單回合文字輸入／輸出，但你也可設定多回合對話：

In [12]:
chat = client.chats.create(model='gemini-2.0-flash', history=[])
response = chat.send_message('哈囉！我叫 Zlork。')
print(response.text)

哈囉 Zlork，很高興認識你！ 有什麼我可以幫忙的嗎？ 或者只是想聊聊天？ 😊



In [13]:
response = chat.send_message('可以跟我分享一些有趣的恐龍知識嗎？')
print(response.text)

沒問題！以下是一些有趣的恐龍知識，希望能讓你覺得有趣：

*   **雷龍（Brontosaurus）差點就不存在了！** 其實雷龍最初的化石只是一個不太完整的迷惑龍骨架。後來，古生物學家認為它們太相似了，所以把雷龍歸類為迷惑龍的一種。但經過漫長的爭論和新的研究，2015年雷龍終於又被確認為獨立的物種了！

*   **恐龍羽毛並不只是裝飾！** 雖然我們總是在電影裡看到皮膚粗糙的恐龍，但其實許多恐龍都有羽毛，而且用途廣泛。有些用來保暖，有些用來求偶，甚至還有用來滑翔的！

*   **暴龍（Tyrannosaurus Rex）可能不是頂級掠食者！** 近年的研究顯示，暴龍可能更多的是以腐肉為食，而不是主動獵殺。雖然牠擁有強大的咬合力，但也許更擅長咬碎骨頭，而不是追逐獵物。

*   **恐龍的壽命超乎想像！** 有些恐龍，比如梁龍（Diplodocus），可能活到 70 甚至 80 歲！這對於生活在恐龍時代的生物來說，算是相當長壽了。

*   **恐龍的便便化石是寶藏！** 古生物學家可以透過研究恐龍的糞便化石（稱為糞化石）來了解牠們的飲食習慣，進而推斷出更多關於恐龍生態的資訊。

*   **最小的恐龍跟鴿子差不多大！** 雖然我們總覺得恐龍都是龐然大物，但其實也有些小巧的恐龍，比如蜂鳥龍（Microraptor），牠們的體型跟鴿子差不多，而且還能滑翔飛行！

*   **恐龍的祖先比恐龍還要早！** 恐龍並不是憑空出現的，牠們的祖先可以追溯到三疊紀時期，比如主龍類（Archosaur），這些生物最終演化成了恐龍、鱷魚和鳥類。

希望這些知識能讓你感到有趣！你想知道更多關於哪方面的恐龍知識呢？例如：特定種類的恐龍、牠們的生活方式、滅絕的原因等等，都可以告訴我喔！



在聊天狀態下，對話內容會持續，你也可確認模型是否記得你的名字：

In [14]:
response = chat.send_message('你還記得我的名字嗎？')
print(response.text)

當然記得！你的名字是 Zlork。 😊



### 選擇模型

Gemini API 提供了 Gemini 模型家族中的多個模型。你可以在 [模型概覽頁面](https://ai.google.dev/gemini-api/docs/models/gemini) 查看各模型功能。

以下範例列出所有可用模型：

In [15]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking

`models.list` 的回應也包含每個模型的額外資訊，如 token 限制與支援參數。以下查詢 `models/gemini-2.0-flash` 的詳細資訊：

In [16]:
from pprint import pprint

for model in client.models.list():
  if model.name == 'models/gemini-2.0-flash':
    pprint(model.to_json_dict())
    break

{'description': 'Gemini 2.0 Flash',
 'display_name': 'Gemini 2.0 Flash',
 'input_token_limit': 1048576,
 'name': 'models/gemini-2.0-flash',
 'output_token_limit': 8192,
 'supported_actions': ['generateContent', 'countTokens'],
 'tuned_model_info': {},
 'version': '2.0'}


## 探索生成參數

### 輸出長度

在使用大型語言模型生成文字時，輸出長度會影響成本與效能。生成更多 token 意味著運算量增加，進而提高能耗、延遲與成本。

若你希望模型在達到特定 token 數後停止生成，可以設定 `max_output_tokens` 參數。這不會使輸出內容更精簡，只是單純截斷生成。


In [20]:
from google.genai import types

short_config = types.GenerateContentConfig(max_output_tokens=200)

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='請撰寫一篇 1000 字論文，探討橄欖在現代社會中的重要性。')

print(response.text)

## 橄欖：現代社會中的文化、經濟與健康瑰寶

橄欖，這種古老的作物，不僅僅是地中海飲食的基石，更在現代社會扮演著多重重要角色。從其豐富的文化底蘊到其在經濟發展中的貢獻，再到其令人矚目的健康益處，橄欖及其衍生產品已經成為全球範圍內不可或缺的一部分。本文將深入探討橄欖在現代社會中的文化、經濟和健康重要性，並展望其未來發展前景。

**一、文化意義：地中海文明的象徵與全球美食的靈感**

橄欖樹與橄欖果在文化上具有深遠的影響，尤其是在地中海地區。幾千年來，橄欖樹被視為和平、智慧、繁榮和長壽的象徵。在古希臘神話中，雅典娜女神用一棵橄欖樹贏


In [21]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=short_config,
    contents='請寫一首短詩，描述橄欖在現代社會中的重要性。')

print(response.text)

碧綠果實，點亮餐桌光，
橄欖油香，滋潤健康忙。
地中海風，吹拂至東方，
和平象徵，滋味細品嚐。



請自行嘗試修改提示內容與參數，觀察輸出結果如何變化。

### 溫度參數

溫度參數控制 token 選擇時的隨機程度。較高溫度會讓模型選擇更多候選 token，輸出結果更具多樣性；溫度設為 0 則代表貪婪解碼，始終選擇機率最高的 token。

雖然溫度不保證完全隨機，但可以「輕微調整」輸出結果。

In [22]:
high_temp_config = types.GenerateContentConfig(temperature=2.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=high_temp_config,
      contents='請隨機選擇一個顏色...（只回傳單一單詞）')

  if response.text:
    print(response.text, '-' * 25)

藍色
 -------------------------
綠
 -------------------------
藍色
 -------------------------
藍色
 -------------------------
藍色
 -------------------------


接著，將溫度設為 0，再試一次。注意雖然結果不完全固定，但會趨向一致。

In [23]:
low_temp_config = types.GenerateContentConfig(temperature=0.0)

for _ in range(5):
  response = client.models.generate_content(
      model='gemini-2.0-flash',
      config=low_temp_config,
      contents='請隨機選擇一個顏色...（只回傳單一單詞）')

  if response.text:
    print(response.text, '-' * 25)

藍色
 -------------------------
藍色
 -------------------------
藍色
 -------------------------
藍色
 -------------------------
藍色
 -------------------------


### Top-P

top-P 參數與溫度類似，用於控制模型輸出的多樣性。

top-P 定義了累積機率達到門檻後停止選擇 token；top-P 為 0 通常等同於貪婪解碼，而 top-P 為 1 則意味著模型會考慮所有 token。

在某些文獻中也會提到 top-K，但在 Gemini 2.0 模型中 top-K 不可調整。

以下範例設定預設值（溫度 1.0，top-p 0.95）來生成故事：

In [27]:
model_config = types.GenerateContentConfig(
    temperature=1.0,
    top_p=0.95,
)

story_prompt = "你是一位創意寫手。請撰寫一個關於一隻冒險中貓咪的短故事。"
response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=story_prompt)

print(response.text)

## 貓咪船長的星辰大海

咪咪是一隻與眾不同的貓。牠不愛在陽光下打盹，也不喜歡追逐紅點。牠的夢想，是星辰大海。

咪咪住在海邊小鎮，每天最愛做的事，就是坐在碼頭邊，看著一艘艘漁船駛向遠方。牠的眼睛裡閃爍著渴望，想像著自己也站在船頭，迎著海風，探索未知的世界。

有一天，機會來了。一艘名為「海鷗號」的貨船即將啟航，前往遙遠的南方島嶼。咪咪偷偷溜上了船，躲在一個裝滿魚乾的木箱裡。

航行開始了。咪咪從木箱裡探出頭，貪婪地呼吸著鹹鹹的海風。牠看著一望無際的藍色，心中充滿了興奮。

船上的水手們很快就發現了咪咪。起初，他們有些驚訝，但很快就被這隻充滿好奇心的小貓咪給迷住了。他們給咪咪取了個綽號：「船長」，因為牠總是喜歡站在船頭，像個真正的船長一樣，眺望遠方。

咪咪很快就適應了船上的生活。牠學會了在搖晃的甲板上保持平衡，學會了分辨不同的海鳥叫聲，甚至還學會了如何從水手們手中偷走美味的魚乾。

航行途中，他們遇到了暴風雨。巨浪拍打著船身，狂風呼嘯著，船隻劇烈搖晃。水手們忙著固定貨物，咪咪也緊緊抓住桅杆，努力不被吹走。

暴風雨過後，天空放晴，海面恢復了平靜。咪咪站在船頭，看著彩虹橫跨天際，心中充滿了劫後餘生的喜悅。

終於，「海鷗號」抵達了南方島嶼。這是一個充滿異國情調的地方，有著茂密的熱帶雨林，色彩鮮豔的鳥類，以及友善的當地居民。

咪咪跳下船，開始探索這個新的世界。牠在雨林中穿梭，追逐蝴蝶，品嚐了從未見過的熱帶水果。牠還結交了一群新的朋友，包括一隻名叫可可的鸚鵡和一隻名叫阿布的猴子。

在南方島嶼待了一段時間後，咪咪開始想家了。牠想念海邊小鎮，想念碼頭，想念那些曾經照顧牠的水手們。

於是，咪咪決定回到「海鷗號」。水手們看到咪咪回來，都非常高興。他們熱烈地歡迎牠，並為牠準備了豐盛的晚餐。

「海鷗號」再次啟航，這次是返回海邊小鎮。咪咪站在船頭，看著南方島嶼漸漸消失在海平面上，心中充滿了不捨，但也充滿了期待。

回到海邊小鎮後，咪咪成為了鎮上的英雄。牠的故事被廣為流傳，人們都稱牠為「貓咪船長」。

咪咪並沒有因此而驕傲。牠仍然喜歡坐在碼頭邊，看著漁船駛向遠方。但現在，牠的眼睛裡不再只有渴望，還有著冒險的記憶，以及對星辰大海的無限憧憬。

咪咪知道，這一次的冒險只是個開始。牠的夢想，是探索更廣闊的世界，是征服更遙遠的星辰大海。而牠，貓咪船長，已經準備好了。



## 提示範例

這一節提供教材中的一些提示範例，供你直接在 API 中試用。你可調整提示文字，看看不同指令、範例或變化下模型的表現。

### 零樣本提示

零樣本提示即直接描述模型需求的提示。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1gzKKgDHwkAvexG5Up0LMtl1-6jKMKe4g"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>

In [31]:
model_config = types.GenerateContentConfig(
    temperature=0.1,
    top_p=1,
    max_output_tokens=5,
)

zero_shot_prompt = """請將電影評論分類為 正面、 中立 或 負面。
評論："Her" 這部電影展現了一種令人不安的研究，揭示了如果 AI 得以無限制進化，
人類將走向何種未來。我真希望能看到更多這樣的經典作品。
情感傾向："""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=model_config,
    contents=zero_shot_prompt)

print(response.text)

情感傾向：**正面


#### Enum 模式

模型訓練目標是生成文字，雖然 Gemini 2.0 模型能很好地遵循指令，但有時候可能會產生超出需求的內容。上述範例中，模型應只回傳標籤，但有時可能會附上額外文字。使用 Enum 模式可以限制模型僅回傳固定選項。


In [33]:
import enum

class Sentiment(enum.Enum):
    POSITIVE = "正面"
    NEUTRAL = "中立"
    NEGATIVE = "負面"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        response_mime_type="text/x.enum",
        response_schema=Sentiment
    ),
    contents=zero_shot_prompt)

print(response.text)

正面


當使用 Enum 模式時，Python SDK 會自動將模型回應轉換成 Python 物件，存放在 `response.parsed` 欄位：

In [34]:
enum_response = response.parsed
print(enum_response)
print(type(enum_response))

Sentiment.POSITIVE
<enum 'Sentiment'>


### 單樣本與少樣本提示

提供一個範例回應即稱為「單樣本提示」，若提供多個範例則為「少樣本提示」。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1jjWkjUSoMXmLvMJ7IzADr_GxHPJVV2bg"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>

In [37]:
few_shot_prompt = """請將顧客的披薩訂單解析成有效的 JSON 格式：

範例：
我想要一個小披薩，搭配起司、番茄醬與義大利臘腸。
JSON 回應：
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "pepperoni"]
}
```

範例：
我可以來一個大披薩，加上番茄醬、羅勒和莫札瑞拉起司。
JSON 回應：
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}
```

訂單：
"""

customer_order = "給我一個大披薩，加上起司和鳳梨"

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        top_p=1,
        max_output_tokens=250,
    ),
    contents=[few_shot_prompt, customer_order])

print(response.text)

```json
{
  "size": "large",
  "type": "normal",
  "ingredients": ["cheese", "pineapple"]
}
```



#### JSON 模式

為了確保輸出嚴格為 JSON 格式，你可以使用 Gemini API 的 [JSON 模式](https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb)，此模式會根據指定結構限制生成內容。


In [41]:
import typing_extensions as typing

class PizzaOrder(typing.TypedDict):
    size: str
    ingredients: list[str]
    type: str

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=0.1,
        response_mime_type="application/json",
        response_schema=PizzaOrder,
    ),
    contents="請給我一個大份的甜點披薩，搭配蘋果和巧克力。")

print(response.text)

{
  "size": "large",
  "ingredients": ["apple", "chocolate"],
  "type": "dessert pizza"
}


### 思維鏈 (Chain of Thought, CoT)
直接提示模型回答可以快速回應，但有時容易出現幻覺或推理錯誤。思維鏈則要求模型先展示推理步驟，再給出最終答案，通常能提高回答正確性，但會增加生成的 token 數量與運算成本。


In [42]:
prompt = """當我 4 歲時，我的伴侶年齡是我的 3 倍。現在我 20 歲，請直接告訴我我的伴侶幾歲？"""
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

print(response.text)

你的伴侶現在 68 歲。



試試加入「一步一步思考」的提示：

In [43]:
prompt = """當我 4 歲時，我的伴侶年齡是我的 3 倍。現在我 20 歲，請一步一步推理，告訴我我的伴侶幾歲？"""
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=prompt)

Markdown(response.text)

好的，我們一步一步來：

1. **計算年齡差：** 當你 4 歲時，你的伴侶 12 歲 (4 歲 * 3 = 12 歲)。
2. **計算年齡差距：** 你的伴侶比你大 8 歲 (12 歲 - 4 歲 = 8 歲)。
3. **計算伴侶現在的年齡：** 因為年齡差距永遠不變，所以現在你的伴侶 28 歲 (20 歲 + 8 歲 = 28 歲)。

**所以，你現在 20 歲，你的伴侶 28 歲。**


### ReAct：結合理性與行動

在此範例中，你將使用 ReAct 提示，讓模型在回答問題時交替輸出「思考」、「行動」與「觀察」步驟。這種結構化提示有助於模型更細緻地解題。以下範例取自 [ReAct GitHub](https://github.com/ysymyth/ReAct)（MIT 授權）。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/18oo63Lwosd-bQ6Ay51uGogB3Wk3H8XMO"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>

In [44]:
model_instructions = """
請解決一個問答任務，交替輸出 思考 (Thought)、行動 (Action) 與 觀察 (Observation) 的步驟。思考用於推理當前狀況，
觀察則是解析行動輸出中的關鍵資訊，而行動可以是以下三種操作之一：
 (1) <search>實體</search>：在 Wikipedia 上搜尋該實體，若存在則返回第一段；若無，則返回相似實體以供再次搜尋。
 (2) <lookup>關鍵字</lookup>：在當前內容中返回包含該關鍵字的下一句，此操作僅做精確匹配，所以請保持搜尋詞簡短。
 (3) <finish>答案</finish>：返回答案並結束任務。
"""

example1 = """Question
音樂人及諷刺作家 Allie Goertz 寫了一首關於《辛普森家庭》角色 Milhouse 的歌曲，請問 Milhouse 是以誰命名的？

Thought 1
問題可簡化為：「《辛普森家庭》角色 Milhouse 是以誰命名？」我只需要搜尋 Milhouse，找出其命名來源。

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten 是《辛普森家庭》中的常客角色，由 Pamela Hayden 配音，由 Matt Groening 創造。

Thought 2
該段文字並未說明 Milhouse 的命名來源，應再搜尋「named after」。

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse 是以美國總統 Richard Nixon 的中間名 Milhous 命名。

Thought 3
因此答案為 Richard Nixon。

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
Colorado 造山運動的東部區域延伸到哪裡？其海拔範圍為何？

Thought 1
我需要搜尋 Colorado 造山運動，找出其東部區域延伸到哪裡，再查該區域的海拔範圍。

Action 1
<search>Colorado orogeny</search>

Observation 1
Colorado 造山運動指的是在 Colorado 及周邊地區發生的造山事件。

Thought 2
未提及東部區域，所以再搜尋「eastern sector」。

Action 2
<lookup>eastern sector</lookup>

Observation 2
東部區域延伸至高原，亦稱為 Central Plains orogeny。

Thought 3
接著搜尋「High Plains」以找出海拔範圍。

Action 3
<search>High Plains</search>

Observation 3
High Plains 指的是大平原中的一個區域。

Thought 4
應改搜尋「High Plains (United States)」。

Action 4
<search>High Plains (United States)</search>

Observation 4
High Plains 指的是美國大平原的一部分，海拔約從 1,800 到 7,000 英尺（550 至 2,130 公尺）。

Thought 5
因此答案為 1,800 到 7,000 英尺。

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

# 你也可以自行增加更多範例，或參考 https://github.com/ysymyth/ReAct/

為了逐步捕捉每個步驟，並忽略模型可能產生的虛假「觀察」步驟，你可以使用 `stop_sequences` 來結束生成過程，停在遇到「Observation」之前。

In [45]:
question = """Question
Transformers NLP 論文中，誰是最年輕的作者？
"""

# 此處設定停止序列，生成內容將停在第一個 "Observation" 前
react_config = types.GenerateContentConfig(
    stop_sequences=["\nObservation"],
    system_instruction=model_instructions + example1 + example2,
)

react_chat = client.chats.create(
    model='gemini-2.0-flash',
    config=react_config,
)

resp = react_chat.send_message(question)
print(resp.text)

Thought 1
首先，我需要找到關於 Transformers NLP 的論文。我將搜尋 "Transformer (model)"。

Action 1
<search>Transformer (model)</search>



接著你可以進一步提供資料給模型，讓其繼續推理。

In [46]:
observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
我們提出了一種全新的網路架構——Transformer，完全依靠注意力機制，捨棄了循環與卷積。
"""
resp = react_chat.send_message(observation)
print(resp.text)

Thought 2
找到了論文，作者有：Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin。現在需要找出最年輕的作者。沒有其他資訊來源，我將搜尋每個作者的出生年份來確定。由於這是一項 NLP 任務，Aidan Gomez 很可能就是論文作者 Aidan N. Gomez。

Action 2
<search>Aidan N. Gomez</search>



該流程會重複進行，直至模型回覆 `<finish>` 指令。你可自行嘗試或參考 [Wikipedia 版 ReAct 範例](https://github.com/google-gemini/cookbook/blob/main/examples/Search_Wikipedia_using_ReAct.ipynb)。


## 思考模式

實驗性的 Gemini Flash 2.0 “Thinking” 模型已訓練出生成「思考過程」的能力，即模型在回答前所經歷的推理步驟。使用思考模式模型，可以在無需特別提示的情況下，得到更高品質的回答。但注意，使用 API 時只會回傳最終答案，若要查看中間思考步驟，請至 AI Studio 試用。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/new_chat?model=gemini-2.0-flash-thinking-exp-01-21"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟思考模式</a>
  </td>
</table>


In [48]:
import io
from IPython.display import Markdown, clear_output

response = client.models.generate_content_stream(
    model='gemini-2.0-flash-thinking-exp',
    contents='Transformers NLP 論文中最年輕的作者是誰？'
)

buf = io.StringIO()
for chunk in response:
    buf.write(chunk.text)
    print(chunk.text, end='')

clear_output()
Markdown(buf.getvalue())

要準確地回答「變形金剛 NLP 論文中最年輕的作者是誰？」這個問題，實際上**非常困難，甚至可以說是不可能準確回答**，原因如下：

1. **缺乏明確的「最年輕」定義和官方記錄:**
   - **年齡隱私:** 學術論文作者的年齡通常不會公開記錄在論文本身或學術資料庫中。追蹤作者的生日並計算發表論文時的年齡是極其繁瑣且不實際的。
   - **論文數量龐大:**  「變形金剛 NLP 論文」是一個非常廣泛的範疇，涵蓋了數以千計甚至數以萬計的論文。要逐一篩選所有論文，找出所有作者，然後再確定他們的年齡，是不現實的。
   - **共同作者問題:** 論文通常有多位作者，如何定義「最年輕的作者」？是指所有作者中最年輕的，還是指在貢獻度上排第一作者但年紀最小的？定義不明確。

2. **「年輕」的定義是相對的:**
   -  對於學術界來說，所謂的「年輕」也可能是相對的。一位在碩士或博士期間發表重要論文的作者，在學術界可能被認為是年輕有為，但他們的實際年齡可能已經二十多歲。

**儘管無法找出確切的最年輕作者，我們可以從以下幾個角度來理解這個問題，並提供一些相關的思考方向：**

* **關注年輕有為的學者:**  與其追求一個無法驗證的「最年輕」頭銜，不如關注那些在 NLP 領域，特別是 Transformer 相關領域展現出卓越才華和潛力的年輕學者。他們可能是在研究生階段就發表重要論文，或者在早期職業生涯中做出突出貢獻。

* **Transformer 論文的作者群體:** 原始的 Transformer 論文 "Attention is All You Need" 的作者團隊來自 Google Brain 和 University of Toronto，他們當時大多是博士生、博士後研究員或年輕的研究科學家。這個團隊的平均年齡相對較輕，但要說誰是最年輕的，仍然需要逐一考證，而且意義不大。

* **年輕作者的意義:**  提問者可能更想了解的是，在如此複雜和前沿的領域，是否有年輕的作者能夠做出重要的貢獻。答案是肯定的。學術界一直都有年輕人嶄露頭角，在 NLP 和 Transformer 領域也不例外。

**總結來說，**  「變形金剛 NLP 論文中最年輕的作者」這個問題，更多的是一個概念性的疑問，而不是一個可以精確回答的事實性問題。 我們無法確定誰是絕對最年輕的，但可以肯定的是，在 NLP 領域，包括 Transformer 研究，一直都有年輕且有才華的作者在做出貢獻。

**建議：** 與其追問「最年輕」，不如關注那些年輕學者在 Transformer NLP 領域的傑出工作，並鼓勵更多年輕人投身於這個充滿活力的研究方向。

## Code 提示

### 生成程式碼

Gemini 模型也能用來生成程式碼、設定檔與腳本，這對學習程式語言或快速產出初稿非常有幫助。  
但請注意，LLM 有時可能產生錯誤或重複訓練資料，務必先閱讀與測試生成的程式碼，並遵守相關授權規定。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1YX71JGtzDjXQkgdes8bP6i3oH5lCRKxv"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>


In [49]:
# Gemini 模型通常會嘮叨，所以如果你只需要程式碼，
# 請明確告知模型僅回傳程式碼，不附加任何解釋。
code_prompt = """
請撰寫一個 Python 函數，用來計算數字的階乘。請只回傳程式碼，不要任何解釋。
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=types.GenerateContentConfig(
        temperature=1,
        top_p=1,
        max_output_tokens=1024,
    ),
    contents=code_prompt)

Markdown(response.text)

```python
def factorial(n):
    """
    Calculates the factorial of a non-negative integer.

    Args:
        n: A non-negative integer.

    Returns:
        The factorial of n, or 1 if n is 0.
        Raises ValueError if n is negative.
    """
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers.")
    if n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result
```

### 程式碼執行

Gemini API 能自動執行生成的程式碼，並回傳執行結果。

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/11veFr_VYEwBWcLkhNLr-maCG0G8sS_7Z"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>


In [51]:

from pprint import pprint

config = types.GenerateContentConfig(
    tools=[types.Tool(code_execution=types.ToolCodeExecution())],
)

code_exec_prompt = """
請生成前 14 個奇數質數，然後計算它們的總和。
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    config=config,
    contents=code_exec_prompt)

for part in response.candidates[0].content.parts:
  pprint(part.to_json_dict())
  print("-----")

{'text': '好的，我將生成前 14 個奇數質數，然後計算它們的總和。\n'
         '\n'
         '首先，我們要找出前 14 個奇數質數。質數是只能被 1 和自身整除的數。奇數質數是除了 2 以外的質數。\n'
         '\n'
         '以下是生成質數並篩選奇數質數的步驟：\n'
         '1. 從 3 開始迭代。\n'
         '2. 檢查當前數字是否為質數。\n'
         '3. 如果是質數，檢查它是否為奇數。\n'
         '4. 如果是奇數質數，添加到列表中。\n'
         '5. 重複步驟 1-4，直到列表中有 14 個數字。\n'
         '\n'}
-----
{'executable_code': {'code': 'def is_prime(n):\n'
                             '    if n <= 1:\n'
                             '        return False\n'
                             '    for i in range(2, int(n**0.5) + 1):\n'
                             '        if n % i == 0:\n'
                             '            return False\n'
                             '    return True\n'
                             '\n'
                             'primes = []\n'
                             'num = 3\n'
                             'while len(primes) < 14:\n'
                             '    if is_prime(num):\n'
                             '  

這個回應包含多個部分：一般文字、生成的程式碼，以及執行程式碼後的結果。你可以分別檢視：

In [52]:
for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    elif part.executable_code:
        display(Markdown(f'```python\n{part.executable_code.code}\n```'))
    elif part.code_execution_result:
        if part.code_execution_result.outcome != 'OUTCOME_OK':
            display(Markdown(f'## 狀態 {part.code_execution_result.outcome}'))
        display(Markdown(f'```\n{part.code_execution_result.output}\n```'))

好的，我將生成前 14 個奇數質數，然後計算它們的總和。

首先，我們要找出前 14 個奇數質數。質數是只能被 1 和自身整除的數。奇數質數是除了 2 以外的質數。

以下是生成質數並篩選奇數質數的步驟：
1. 從 3 開始迭代。
2. 檢查當前數字是否為質數。
3. 如果是質數，檢查它是否為奇數。
4. 如果是奇數質數，添加到列表中。
5. 重複步驟 1-4，直到列表中有 14 個數字。



```python
def is_prime(n):
    if n <= 1:
        return False
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            return False
    return True

primes = []
num = 3
while len(primes) < 14:
    if is_prime(num):
        primes.append(num)
    num += 2  # Only check odd numbers

print(f'{primes=}')

import numpy as np
sum_of_primes = np.sum(primes)
print(f'{sum_of_primes=}')

```

```
primes=[3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
sum_of_primes=np.int64(326)

```

前 14 個奇數質數是 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43 和 47。它們的總和是 326。


### 解釋程式碼

Gemini 模型也能解釋程式碼。例如，你可傳入一個 [bash 腳本](https://github.com/magicmonty/bash-git-prompt) 並提問：

<table align=left>
  <td>
    <a target="_blank" href="https://aistudio.google.com/prompts/1N7LGzWzCYieyOf_7bAG4plrmkpDNmUyb"><img src="https://ai.google.dev/site-assets/images/marketing/home/icon-ais.png" style="height: 24px" height=24/> 在 AI Studio 開啟</a>
  </td>
</table>

In [53]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

explain_prompt = f"""
請以簡單明瞭的方式說明這個檔案的作用。它是什麼？我為何要使用它？

```
{file_contents}
```
"""

response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=explain_prompt)

Markdown(response.text)

這個檔案是一個 **Bash 腳本**，設計用來 **強化你的終端機提示符號，讓你更容易掌握 Git 儲存庫的狀態**。

**它是什麼？**

*   **Bash 腳本：**  它是一系列命令，由 Bash (或其他相容的 shell，如 Zsh) 解譯並執行。
*   **Git Prompt 增強工具：** 它主要的作用是讓你目前的目錄是一個 Git 儲存庫時，在你的命令提示字元中顯示 Git 的相關資訊。

**它做什麼？**

1.  **顯示 Git 狀態：**  告訴你目前的 Git 分支、是否已修改、是否有未提交的變更、是否與遠端儲存庫同步等等。
2.  **可客製化：**  你可以設定顏色、符號和要顯示的資訊，根據你的喜好調整提示符號。
3.  **使用主題：** 可以使用預設主題或者自訂主題。
4.  **支援虛擬環境：** 可以同時顯示 Python 的虛擬環境名稱。

**為何要使用它？**

*   **提高效率：**  一眼就能看到 Git 狀態，不用每次都輸入 `git status`。
*   **減少錯誤：**  在提交之前，能提醒你是否有未提交的變更。
*   **更好的工作流程：**  在終端機中就能更清楚地了解 Git 儲存庫的狀態，使你更專注在工作上。
*   **增強可視性：** 彩色提示符號能更快速地識別資訊。

**簡單來說：**

這個檔案就像一個外掛程式，幫你的終端機提示符號顯示重要的 Git 資訊。如果你經常使用 Git，它可以讓你更有效率地工作，並減少 Git 相關的錯誤。

**如何使用它？**

你通常需要將這個腳本複製到你的系統上，並在你的 shell 配置文件（例如 `.bashrc` 或 `.zshrc`）中 `source` 它，以便在每次啟動終端機時自動執行。 腳本本身包含一個函數 `gp_install_prompt`，可以用來安裝它。


回應內容可能會說明：  
這個 `git-prompt.sh` 是一個 shell 腳本，設計來美化你在 Git 使用時的命令提示符，直接顯示目前 Git 狀態（例如分支、檔案變動等）。簡而言之，它是一個提升效率與視覺效果的工具。

## 進一步學習

若想更深入了解提示工程，請參考以下資源：

* 閱讀今日教材附上的白皮書
* 試用筆記本上方推薦的應用程式（如 [TextFX](https://textfx.withgoogle.com/)、[SQL Talk](https://sql-talk-r5gdynozbq-uc.a.run.app/) 及 [NotebookLM](https://notebooklm.google/)）
* 閱讀 Gemini API 文件中的 [提示工程入門](https://ai.google.dev/gemini-api/docs/prompting-intro)
* 探索 Gemini API 的 [提示範例庫](https://ai.google.dev/gemini-api/prompts) 並在 AI Studio 試用
* 參考 Gemini API Cookbook 的 [範例](https://github.com/google-gemini/cookbook/blob/main/examples/) 與 [快速入門](https://github.com/google-gemini/cookbook/blob/main/quickstarts/)

另外，請別忘了參加 Day 3 的 codelab，屆時會介紹更進階的提示結合程式碼執行技巧。

In [3]:
# Copyright 2025 Google LLC.

# @title 根據 Apache License, Version 2.0（以下簡稱「授權條款」）取得授權；
# 除非遵循授權條款，否則不得使用此檔案。
# 你可以從以下網址取得授權條款副本：
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# 除非適用法律要求或書面同意，本軟體皆按「現狀」提供，
# 不附帶任何形式的明示或暗示保證。
# 請參閱授權條款以了解具體權限與限制。